In [1]:
import numpy as np
import pandas as pd
import sys
from datetime import datetime
import skmap_bindings
from pathlib import Path
import time
import os
import random
os.environ['PROJ_LIB'] = '/opt/conda/share/proj/'

gdal_opts = {
 'GDAL_HTTP_VERSION': '1.0',
 'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': '.tif',
}
n_threads = 96
bands_list = [1,]
x_off = 0
y_off = 0
x_size, y_size = (4004, 4004)
nodata_val = 255
compression_command = f"gdal_translate -a_nodata {nodata_val} -co COMPRESS=deflate -co TILED=TRUE -co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 -co NUM_THREADS=8"
date = '20220701_20220831'
tmp_write_dir = f'../tmp_{date}'
band_scaling = 0.004
result_scaling = 125.
result_offset = 125.
climping = [0., 250.]

In [2]:
with open('../global_tiles_s1.csv') as file:
    tiles = [f'{line.rstrip()}' for line in file]

# tiles = ['110E_00S', '112E_00S', '100E_00S',]    
n_tiles = len(tiles)

In [3]:
# for i in range(0, n_tiles, n_threads):
# for i in range(7968, 16000, n_threads):
for i in range(15000, n_tiles, n_threads):
    tiles_chunk = tiles[i:min(i+n_threads, n_tiles)]
    n_chunk = len(tiles_chunk)
    print(f"Processing chunk from {i} to {min(i+n_threads, n_tiles)}", flush=True)
    start = time.time()
    files_mask = [f'../../GPW/global_landmask/gpw_landmask_{tile}_ultimate.tif' for tile in tiles_chunk]
    files_out = [f'ndvi_{date}_{tile}_landmasked' for tile in tiles_chunk]
    files_red = [f'http://192.168.49.30:8333/prod-landsat-ard2/{tile}/seasconv/red_glad.SeasConv.ard2_m_30m_s_{date}_go_epsg.4326_v20230908.tif' for tile in tiles_chunk]
    files_nir = [f'http://192.168.49.30:8333/prod-landsat-ard2/{tile}/seasconv/nir_glad.SeasConv.ard2_m_30m_s_{date}_go_epsg.4326_v20230908.tif' for tile in tiles_chunk]
    mask = np.empty((n_chunk, x_size * y_size), dtype=np.float32)
    mask_idx = range(n_chunk)
    nir_red_ndvi = np.empty((3*n_chunk, x_size * y_size), dtype=np.float32)
    nir_idx = range(n_chunk)
    red_idx = range(n_chunk,2*n_chunk)
    ndvi_idx = range(2*n_chunk,3*n_chunk)

    skmap_bindings.readData(mask, n_threads, files_mask, mask_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)
    skmap_bindings.readData(nir_red_ndvi, n_threads, files_nir, nir_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)
    skmap_bindings.readData(nir_red_ndvi, n_threads, files_red, red_idx, x_off, y_off, x_size, y_size, bands_list, gdal_opts)

    skmap_bindings.computeNormalizedDifference(nir_red_ndvi, n_threads,
                                                nir_idx, red_idx, ndvi_idx,
                                                band_scaling, band_scaling, result_scaling, result_offset, climping)

    skmap_bindings.maskData(nir_red_ndvi, n_threads, ndvi_idx, mask, 0., 255.)

    nir_red_ndvi[ndvi_idx,:][mask[:,:] == 0.] = 255.
    skmap_bindings.writeByteData(nir_red_ndvi, n_threads, gdal_opts, files_red, tmp_write_dir, files_out, ndvi_idx,
        x_off, y_off, x_size, y_size, nodata_val, compression_command)
    # print(f"Processed files in {time.time() - start:.2f} s", flush=True)

Processing chunk from 15000 to 15096
Processing chunk from 15096 to 15192
Processing chunk from 15192 to 15288
Processing chunk from 15288 to 15384
Processing chunk from 15384 to 15480
Processing chunk from 15480 to 15576
Processing chunk from 15576 to 15672
Processing chunk from 15672 to 15768
Processing chunk from 15768 to 15864
Processing chunk from 15864 to 15960
Processing chunk from 15960 to 16056
Processing chunk from 16056 to 16152
Processing chunk from 16152 to 16248
Processing chunk from 16248 to 16344
Processing chunk from 16344 to 16440
Processing chunk from 16440 to 16536
Processing chunk from 16536 to 16632
Processing chunk from 16632 to 16728
Processing chunk from 16728 to 16824
Processing chunk from 16824 to 16920
Processing chunk from 16920 to 17016
Processing chunk from 17016 to 17112
Processing chunk from 17112 to 17208
Processing chunk from 17208 to 17304
Processing chunk from 17304 to 17400
Processing chunk from 17400 to 17496
Processing chunk from 17496 to 17592
P